In [ ]:
'''

用戶菜單功能介紹

    用戶能透過點擊菜單，進行我方希冀之業務功能。
    
流程
    準備菜單的圖面設定檔
    讀取安全設定檔上的參數
    將菜單設定檔傳給Line
    對Line上傳菜單照片
    檢視現有的菜單
    將菜單與用戶做綁定
    將菜單與用戶解除綁定
    刪除菜單

'''

In [ ]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": false,
  "name": "流程",
  "chatBarText": "流程",
  "areas": [
    {
      "bounds": {
        "x": 30,
        "y": 88,
        "width": 526,
        "height": 810
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "folder=啟動"
      }
    },
    {
      "bounds": {
        "x": 813,
        "y": 41,
        "width": 654,
        "height": 853
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "folder=area"
      }
    },
    {
      "bounds": {
        "x": 1695,
        "y": 54,
        "width": 608,
        "height": 815
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "folder=fundraising"
      }
    },
    {
      "bounds": {
        "x": 1686,
        "y": 877,
        "width": 659,
        "height": 806
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "folder=申請"
      }
    },
    {
      "bounds": {
        "x": 795,
        "y": 907,
        "width": 679,
        "height": 761
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "folder=operation"
      }
    },
    {
      "bounds": {
        "x": 130,
        "y": 1068,
        "width": 455,
        "height": 312
      },
      "action": {
        "type": "postback",
        "text": "",
        "data": "data=main"
      }
    }
  ]
}
"""

In [ ]:
'''

讀取安全檔案內的字串，以供後續程式碼調用

'''
import json
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))


In [ ]:
'''

用channel_access_token創建line_bot_api，預備用來跟Line進行溝通

'''

from linebot import (
    LineBotApi, WebhookHandler
)

line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))


In [ ]:
'''

載入前面的圖文選單設定，

並要求line_bot_api將圖文選單上傳至Line
    

'''

from linebot.models import RichMenu
import requests

#讀取圖文選單設定檔，變成json
menuJson=json.loads(menuRawData)

#請line_bot_api跟line說創建圖文選單，創建的時候必須給json檔參數設定
lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(menuJson))

# 打印結果
print(lineRichMenuId)



In [ ]:
'''

將先前準備的菜單照片，以Post消息寄發給Line

    載入照片
    要求line_bot_api，將圖片傳到先前的圖文選單id


'''

#載入照片
uploadImageFile=open("../images/processing.jpg",'rb')

# 上傳圖文選單的照片給line
setImageResponse = line_bot_api.set_rich_menu_image(lineRichMenuId,'image/jpeg',uploadImageFile)

# 打印結果
print(setImageResponse)


In [ ]:
'''

將選單綁定到特定用戶身上
    取出上面得到的菜單Id及用戶id
    要求line_bot_api告知Line，將用戶與圖文選單做綁定

'''

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}

# 透過line_bot_api，請line幫我們做綁定
linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["self_user_id"], lineRichMenuId)

print(linkResult)


In [ ]:
'''

檢視用戶目前所綁定的菜單
    取出用戶id，並告知line_bot_api，
    line_bot_api傳回用戶所綁定的菜單
    印出

'''

#  https://api.line.me/v2/bot/user/{userId}/richmenu

# 請line_bot_api去跟line查詢，該用戶所屬的圖文選單
rich_menu_id = line_bot_api.get_rich_menu_id_of_user(secretFileContentJson["self_user_id"])

# 打印id
print(rich_menu_id)



In [ ]:
'''

解除選單與特定用戶的綁定
    取出用戶id，並告知line_bot_api，
    line_bot_api解除用戶所綁定的菜單
'''

# 請line_bot_api解除指定用戶的圖文選單
lineUnregisterUserMenuResponse=line_bot_api.unlink_rich_menu_from_user(secretFileContentJson["self_user_id"])
print(lineUnregisterUserMenuResponse)


In [ ]:
'''

檢視帳號內，有哪些選單
    要求line_bot_api，向line查詢我方的圖文選單列表
    打印

'''

rich_menu_list = line_bot_api.get_rich_menu_list()
for rich_menu in rich_menu_list:
    print(rich_menu.rich_menu_id)